# Thin Region Dataset - Stats and tranformations

# Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.express as px



from PIL import Image
import torch
from torchvision.transforms import v2

import os

from concurrent.futures import ThreadPoolExecutor

# Functions

In [2]:

def img_opener(img_path):
    """
    Opens images from a folder and returns a list of PIL images
    """
    imgs = []
    filenames_list = []
    filenames = sorted(os.listdir(img_path))
    
    for filename in filenames:
        file_path = os.path.join(img_path, filename)
        img = Image.open(file_path)
        imgs.append(img)
        filenames_list.append(filename)
    
    return imgs, filenames_list

In [3]:
def get_image_dimensions(imgs, filenames_list=None):
    """
    Returns the dimensions of the images
    
    Parameters
    ----------
        imgs (list): A list of PIL.Image objects representing the images.
    
    Returns
    -------
        pandas.DataFrame: A DataFrame containing the width, height, and aspect ratio of each image.
    """
    dimensions = []
    
    for img in imgs:
        width, height = img.size
        dimensions.append((width, height))
    
    df = pd.DataFrame(dimensions, columns=['width', 'height'])
    df['aspect_ratio'] = df['width'] / df['height']
    if filenames_list:
        df['filename'] = filenames_list
    
    df = df[['filename', 'width', 'height', 'aspect_ratio']]
    
    return df

In [4]:
def get_intensity_stats(imgs):
    """
    Returns the mean and standard deviation of the pixel intensities of the images.
    
    Parameters
    ----------
        imgs (list): A list of PIL.Image objects representing the images.
    
    Returns
    -------
        pandas.DataFrame: A DataFrame containing the mean and standard deviation of the pixel intensities of each image.
    """
    stats = []
    
    for img in imgs:
        arr = np.array(img)
        mean = arr.mean()
        std = arr.std()
        max_val = arr.max()
        min_val = arr.min()
        stats.append((mean, std, max_val, min_val))
    
    df = pd.DataFrame(stats, columns=['mean', 'std', 'max', 'min'])
    
    return df

In [5]:
def calculate_stats(img):
    """
    Calcula as estatísticas de intensidade de pixels de uma única imagem usando PyTorch.
    
    Parameters
    ----------
        img (PIL.Image): Um objeto PIL.Image representando a imagem.
    
    Returns
    -------
        tuple: Uma tupla contendo a média, o desvio padrão, o valor máximo e o valor mínimo das intensidades de pixels.
    """
    
    img = np.array(img)
    img = torch.from_numpy(img).permute(2, 0, 1)
    
    arr = torch.tensor(img, dtype=torch.float32).cuda()
    mean = torch.mean(arr)
    std = torch.std(arr)
    max_val = torch.max(arr)
    min_val = torch.min(arr)
    return float(mean), float(std), float(max_val), float(min_val)

def get_batch_stats(img_batch):
    """
    Calcula as estatísticas de intensidade de pixels de um lote (batch) de imagens usando PyTorch.
    
    Parameters
    ----------
        img_batch (list): Uma lista de objetos PIL.Image representando as imagens no lote.
    
    Returns
    -------
        list: Uma lista de tuplas contendo a média, o desvio padrão, o valor máximo e o valor mínimo das intensidades de pixels.
    """
    return [calculate_stats(img) for img in img_batch]


def get_intensity_stats_parallel(imgs, num_threads=4, batch_size=10):
    """
    Retorna a média e o desvio padrão das intensidades de pixels das imagens de forma paralela usando PyTorch na GPU.
    
    Parameters
    ----------
        imgs (list): Uma lista de objetos PIL.Image representando as imagens.
        num_threads (int): O número de threads paralelos a serem utilizados.
        batch_size (int): O tamanho do lote (batch) para processamento em lote.
    
    Returns
    -------
        pandas.DataFrame: Um DataFrame contendo a média e o desvio padrão das intensidades de pixels de cada imagem.
    """
    stats = []
    batches = [imgs[i:i + batch_size] for i in range(0, len(imgs), batch_size)]

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        for batch_stats in executor.map(get_batch_stats, batches):
            stats.extend(batch_stats)

    df = pd.DataFrame(stats, columns=['mean', 'std', 'max', 'min'])
    return df






# Image stats

## Dimensions

In [6]:
img_path = '/home/wesleygalvao/Documents/repositorios/thin-object-selection/data/ThinObject5K_fine_tuning/images'
label_path = '/home/wesleygalvao/Documents/repositorios/thin-object-selection/data/ThinObject5K_fine_tuning/masks'

In [7]:
# Load images
imgs, filenames_list = img_opener(img_path)
# Get image dimensions
df_dimensions = get_image_dimensions(imgs, filenames_list)
df_dimensions.head(5)

,filename,width,height,aspect_ratio
0,air_pump_PNG1.jpg,368,368,1.00000
1,air_pump_PNG12.jpg,458,458,1.00000
2,air_pump_PNG13b.jpg,412,320,1.28750
3,air_pump_PNG15a.jpg,414,320,1.29375
4,air_pump_PNG15b.jpg,414,320,1.29375


In [30]:
df_dimensions.describe(percentiles=[0.25, 0.3, 0.4, 0.5, 0.75])

,width,height,aspect_ratio,intensity_mean,intensity_std,intensity_max,intensity_min
count,4598.000000,4598.000000,4598.000000,4598.000000,4598.000000,4598.000000,4598.000000
mean,1317.426925,1180.739669,1.355584,111.696465,62.012612,254.821444,0.105481
std,955.718994,914.174691,0.998667,34.068324,13.286629,2.477236,1.691952
min,39.000000,32.000000,0.080866,15.281269,9.056536,154.000000,0.000000
25%,600.000000,500.000000,0.913525,87.986511,53.905164,255.000000,0.000000
30%,677.200000,576.200000,1.000000,93.423719,55.832006,255.000000,0.000000
40%,900.000000,748.800000,1.000000,102.334869,59.492245,255.000000,0.000000
50%,1003.500000,940.000000,1.051969,110.750534,62.452971,255.000000,0.000000
75%,1876.000000,1600.000000,1.509612,133.236919,70.882904,255.000000,0.000000
max,11936.000000,8312.000000,12.992593,236.264099,106.361855,255.000000,77.000000


In [29]:
fig1 = px.histogram(df_dimensions, x='width', nbins=250)
fig2 = px.histogram(df_dimensions, x='height', nbins=250)

fig = make_subplots(rows=1, cols=2, subplot_titles=['Width', 'Height'])

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_layout(title_text='Dimensions histograms', showlegend=False)

fig.update_xaxes(title_text='Width', row=1, col=1)
fig.update_xaxes(title_text='Height', row=1, col=2)
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_yaxes(title_text='Count', row=1, col=2)

fig.update_layout(
    title_text='Distribution of Width and Height ',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=500,
)

fig.show()


In [10]:
fig3 = px.histogram(df_dimensions, x='aspect_ratio', title='Apect Ratio histogram', nbins=150)
fig3.update_layout(
    title_x=0.5,
    autosize=False,
    width=700,
    height=500,
)
fig3.show()

## Intensity mean and std

In [11]:
%%time
df_intensity_stats = get_intensity_stats_parallel(imgs, num_threads=12, batch_size=100)

/tmp/ipykernel_74834/4162824829.py:17: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



CPU times: user 6min 40s, sys: 18min 49s, total: 25min 29s
Wall time: 1min 41s


In [12]:
df_intensity_stats.head()

,mean,std,max,min
0,82.481026,78.356850,255.0,0.0
1,110.099121,64.671974,255.0,0.0
2,117.845680,57.127831,255.0,0.0
3,85.679634,71.615105,255.0,0.0
4,76.750664,51.371170,255.0,0.0


In [21]:
111.696465	/255

0.4380253529411765

In [20]:
df_intensity_stats.describe()

,mean,std,max,min
count,4598.000000,4598.000000,4598.000000,4598.000000
mean,111.696465,62.012612,254.821444,0.105481
std,34.068324,13.286629,2.477236,1.691952
min,15.281269,9.056536,154.000000,0.000000
25%,87.986511,53.905164,255.000000,0.000000
50%,110.750534,62.452971,255.000000,0.000000
75%,133.236919,70.882904,255.000000,0.000000
max,236.264099,106.361855,255.000000,77.000000


In [13]:
df_dimensions['intensity_mean'] = df_intensity_stats['mean']
df_dimensions['intensity_std'] = df_intensity_stats['std']
df_dimensions['intensity_max'] = df_intensity_stats['max']
df_dimensions['intensity_min'] = df_intensity_stats['min']

df_dimensions.head(5)

,filename,width,height,aspect_ratio,intensity_mean,intensity_std,intensity_max,intensity_min
0,air_pump_PNG1.jpg,368,368,1.00000,82.481026,78.356850,255.0,0.0
1,air_pump_PNG12.jpg,458,458,1.00000,110.099121,64.671974,255.0,0.0
2,air_pump_PNG13b.jpg,412,320,1.28750,117.845680,57.127831,255.0,0.0
3,air_pump_PNG15a.jpg,414,320,1.29375,85.679634,71.615105,255.0,0.0
4,air_pump_PNG15b.jpg,414,320,1.29375,76.750664,51.371170,255.0,0.0


In [14]:
path = '/home/wesleygalvao/Documents/repositorios/thin-object-selection/data/ThinObject5K_fine_tuning/'
df_dimensions.to_csv(path+'/images_stats.csv', index=False)
df_dimensions.to_csv('./images_stats.csv', index=False)

In [15]:
df_stats = pd.read_csv('./images_stats.csv')
df_stats

,filename,width,height,aspect_ratio,intensity_mean,intensity_std,intensity_max,intensity_min
0,air_pump_PNG1.jpg,368,368,1.000000,82.481026,78.356850,255.0,0.0
1,air_pump_PNG12.jpg,458,458,1.000000,110.099121,64.671974,255.0,0.0
2,air_pump_PNG13b.jpg,412,320,1.287500,117.845680,57.127831,255.0,0.0
3,air_pump_PNG15a.jpg,414,320,1.293750,85.679634,71.615105,255.0,0.0
4,air_pump_PNG15b.jpg,414,320,1.293750,76.750664,51.371170,255.0,0.0
...,...,...,...,...,...,...,...,...
4593,zipper_PNG17.jpg,432,600,0.720000,100.624084,65.693939,255.0,0.0
4594,zipper_PNG20.jpg,1472,1800,0.817778,97.601990,69.332748,255.0,0.0
4595,zipper_PNG49.jpg,588,1360,0.432353,145.518616,37.127983,255.0,0.0
4596,zipper_PNG55.jpg,612,612,1.000000,108.179688,75.540749,255.0,0.0


In [33]:
df_stats['min_size'] = df_stats[['width', 'height']].min(axis=1)
df_stats['max_size'] = df_stats[['width', 'height']].max(axis=1)

In [34]:
df_stats.head(5)

,filename,width,height,aspect_ratio,intensity_mean,intensity_std,intensity_max,intensity_min,min_size,max_size
0,air_pump_PNG1.jpg,368,368,1.00000,82.481026,78.356850,255.0,0.0,368,368
1,air_pump_PNG12.jpg,458,458,1.00000,110.099121,64.671974,255.0,0.0,458,458
2,air_pump_PNG13b.jpg,412,320,1.28750,117.845680,57.127831,255.0,0.0,320,412
3,air_pump_PNG15a.jpg,414,320,1.29375,85.679634,71.615105,255.0,0.0,320,414
4,air_pump_PNG15b.jpg,414,320,1.29375,76.750664,51.371170,255.0,0.0,320,414


In [35]:
px.scatter(df_stats, x='min_size', y='max_size', title='Scatter plot of min and max size')

### Sanity check

Select individual images and check if the stats are consistent with the dataset stats

#### OK

In [16]:
def sanity_check(img_path):
    """
    Opens images from a folder and returns a list of PIL images
    """


    img = Image.open(img_path)   
    width, height = img.size
    aspect_ratio = round(width / height , 2)
    arr = np.array(img)
    mean = round(arr.mean(), 2)
    std = round(arr.std(), 2)
    max_val = round(arr.max(), 2)
    min_val = round(arr.min(), 2)
    
    
    print(f'width: {width}, height: {height}, aspect_ratio: {aspect_ratio}, mean: {mean}, std: {std}, max: {max_val}, min: {min_val}')

In [17]:
img_path = '/home/wesleygalvao/Documents/repositorios/thin-object-selection/data/ThinObject5K_fine_tuning/images'
sanity_check(img_path + '/motorcycle_PNG3159.jpg')

width: 1124, height: 677, aspect_ratio: 1.66, mean: 134.32, std: 100.04, max: 255, min: 0


### Intensity distribution

In [18]:
fig4 = px.histogram(df_stats, x='intensity_mean', title='intensity mean histogram', nbins=150)
fig4.update_layout(
    title_x=0.5,
    autosize=False,
    width=700,
    height=500,
)
fig4.show()

In [19]:
def image_resize(imgs, size_smaller, size_larger):
    """
    Resize images to 256x256
    """
    for i, img in enumerate(imgs):
        
        
        if img.size[0] < size_smaller or img.size[1] < size_smaller:
            img = v2.functional.resize(
                img, 
                size=size_smaller, 
                max_size=size_larger
                interpolation=v2.InterpolationMode.BILINEAR, 
                antialias=True
            )
        
            imgs[i] = img
            print(f'Counter: {i+1} - Images resized')
        
    
    return imgs
        

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1060612657.py, line 12)

In [28]:
512*1.1484375

588.0

In [ ]:
%%time
size_smaller = 512*1.1484375
size_larger = size_smaller*2
imgs_resized = image_resize(imgs, size_smaller, size_larger)

TypeError: object of type 'float' has no len()

In [ ]:
for i, img in enumerate(imgs):
    print(f'Image {i+1} - {img.size}')

Image 1 - (368, 368)
Image 2 - (458, 458)
Image 3 - (412, 320)
Image 4 - (414, 320)
Image 5 - (414, 320)
Image 6 - (1280, 735)
Image 7 - (1000, 700)
Image 8 - (700, 1693)
Image 9 - (1000, 667)
Image 10 - (350, 396)
Image 11 - (1057, 2284)
Image 12 - (1024, 1024)
Image 13 - (876, 1955)
Image 14 - (800, 334)
Image 15 - (1800, 548)
Image 16 - (1920, 1080)
Image 17 - (1580, 1581)
Image 18 - (256, 256)
Image 19 - (400, 289)
Image 20 - (1920, 825)
Image 21 - (398, 150)
Image 22 - (941, 595)
Image 23 - (600, 601)
Image 24 - (1442, 1135)
Image 25 - (580, 368)
Image 26 - (1905, 2130)
Image 27 - (1024, 1201)
Image 28 - (1616, 2223)
Image 29 - (936, 1298)
Image 30 - (1753, 2384)
Image 31 - (1514, 2179)
Image 32 - (1118, 1500)
Image 33 - (768, 1074)
Image 34 - (1457, 2120)
Image 35 - (535, 587)
Image 36 - (1028, 1398)
Image 37 - (920, 920)
Image 38 - (400, 353)
Image 39 - (900, 812)
Image 40 - (900, 812)
Image 41 - (900, 812)
Image 42 - (1995, 1800)
Image 43 - (2128, 1784)
Image 44 - (475, 429)
Im